In [1]:
import pandas as pd

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)

# **Baixando os arquivos**

O script abaixo foi provido pelo professor Xéxeo e ele é responsável por baixar o arquivo necessário para a tarefa e incluí-lo em uma pasta contendo o csv necessário.


In [2]:
# prompt: faça um programa que baixe um arquivo zip do google drive a partir de um link específico e
# coloque o diretório raiz

# Import necessary libraries
import os
from google.colab import drive

# Define the URL of the zip file
url = 'https://github.com/LINE-PESC/Dados-Exercicios-Texto/raw/main/Base%20para%20usar%20no%20trabalho%20CysticFibrosis2-20240401.zip'

# Get the filename from the URL
filename = url.split('/')[-1]

# Create a directory to store the downloaded file
!mkdir data

# Download the zip file
!wget --no-check-certificate {url} -O {filename}

# Unzip the downloaded file
!unzip -j {filename} -d data


--2024-05-11 20:19:55--  https://github.com/LINE-PESC/Dados-Exercicios-Texto/raw/main/Base%20para%20usar%20no%20trabalho%20CysticFibrosis2-20240401.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/LINE-PESC/Dados-Exercicios-Texto/main/Base%20para%20usar%20no%20trabalho%20CysticFibrosis2-20240401.zip [following]
--2024-05-11 20:19:56--  https://raw.githubusercontent.com/LINE-PESC/Dados-Exercicios-Texto/main/Base%20para%20usar%20no%20trabalho%20CysticFibrosis2-20240401.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1548260 (1.5M) [application/zip]
Saving to: ‘Base%20para%20usar%20no%20tr

# **Lendo e processando arquivos xml**
Primeiramente precisamos ler o arquivo cfquery.xml de acordo com o formato estabelecido no arquivo cfc2-query.dtd . Então eu criei uma função especificamente para isso, já implementando o sistema de logging. Depois criaremos consultas com os textos extraídos do arquivo cfquery.xml, processaremos os textos, criaremos um arquivo csv com as consultas processadas e um arquivo csv com os resultados esperados.

In [3]:
import os
import xml.etree.ElementTree as ET
import csv
import logging
import time
import re

#abaixo está a biblioteca responsável pela maioria do pré-processamento de texto
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

#Configuração inicial do logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

#Cria o arquivo de configuração PC.CFG com caminhos predefinidos.
def create_pc_cfg(path):
    logging.info("Arquivo PC.CFG sendo criado")
    cfg_content = """LEIA=cfquery.xml
    CONSULTAS=processed_queries.csv
    ESPERADOS=expected_results.csv"""

    cfg_path = os.path.join(path, 'PC.CFG')
    with open(cfg_path, 'w') as cfg_file:
        cfg_file.write(cfg_content)
    logging.info("Arquivo PC.CFG criado com sucesso")

#Lê o arquivo de configuração e retorna um dicionário com as configurações.
def read_config(path):
    logging.info("Lendo arquivo de configuração")
    config = {}
    with open(path, 'r') as file:
        for line in file:
            key, value = line.strip().split('=')
            config[key.strip()] = value.strip()
    return config


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [4]:
#Esse é o caminho padrão que iremos usar ao longo desse notebook
path = '/content/data'
data = pd.read_xml(path + "/cfquery.xml")
data.head(2)

,QueryNumber,QueryText,Results,Records
0,1,What are the effects of calcium on the physical properties of mucus\n from CF patients?,34,NaN
1,2,Can one distinguish between the effects of mucus hypersecretion and\n infection on the submucosal glands of the respiratory tract in CF?,7,NaN


In [5]:
'''
Normaliza, tokeniza, remove stop words, aplica lematização no texto,
e retorna as palavras em letras maiúsculas.
'''
def preprocess_text(text):
    #Normaliza e converte para minúsculas para processamento uniforme
    text = text.lower()
    #Remove caracteres especiais
    text = re.sub(r'[^a-z0-9\s]', '', text)
    #Tokenização
    tokens = word_tokenize(text)
    # Carrega a lista de stop words
    stop_words = set(stopwords.words('english'))
    #Lematização e remoção de stop words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    #Converte tokens de volta para string, mas em maiúsculas
    processed_text = ' '.join(tokens).upper()
    return processed_text

# Função para processar o XML
def processar_xml(path):
    #armazena os dados das consultas
    consultas = []
    #caminhos dos arquivos XML a serem processados
    arquivos_para_processar = []

    '''
      pd.read_xml não está funcionando corretamente, então estou adotando outra estratégia
      para ler o xml.
    '''
    #Abaixo estamos acessando todos os xmls válidos de um diretório válido
    if os.path.isdir(path):
        #Nesse caso, só temos um xml para ler, mas essa função pode ser usada em outros casos
        arquivos_para_processar = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.xml')]
    #Verifica se o caminho fornecido é um arquivo XML
    elif os.path.isfile(path) and path.endswith('.xml'):
        #Se for um arquivo, adiciona o caminho do arquivo à lista de arquivos xml. Nesse caso é um arquivo só
        arquivos_para_processar.append(path)
    #Se o caminho não for nem um diretório nem um arquivo XML, registra um erro
    else:
        logging.error("Caminho fornecido não é um diretório ou arquivo XML válido.")
        return consultas  # Retorna a lista vazia de consultas caso não tenhamos um xml válido

    #Processa os xmls
    for arquivo in arquivos_para_processar:
        logging.info(f"Processando arquivo {arquivo}")
        tree = ET.parse(arquivo)
        #Obtém a raiz do documento XML
        root = tree.getroot()

        #Procura por todas as tags 'QUERY' dentro do documento XML
        for query in root.findall('QUERY'):
            #Extrai informações de número e texto da consulta
            consulta_info = {
                'QueryNumber': query.find('QueryNumber').text,
                'QueryText': query.find('QueryText').text,
                'Records': []  # Inicializa uma lista para armazenar registros relacionados à consulta
            }

            #Itera sobre cada 'Item' dentro da tag 'Records'
            for item in query.find('Records').findall('Item'):
                #Cria um dicionário com o número do documento e a pontuação de relevância
                doc_info = {
                    'DocNumber': item.text,
                    'Score': item.get('score')
                }
                #Adiciona o dicionário de informações do documento à lista de registros da consulta
                consulta_info['Records'].append(doc_info)
            consultas.append(consulta_info)

    #Retorna a lista de consultas contendo as informações extraídas
    return consultas

# Função para gerar arquivos CSV para consultas processadas
def generate_processed_queries(consultas, path):
    logging.info(f"Arquivo de consultas processadas começou a ser criado no caminho: {path}.")
    with open(path, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['QueryNumber', 'QueryText']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')
        writer.writeheader()
        for consulta in consultas:
            logging.info(f"Consulta {consulta} está pré-processando os textos.")
            processed_text = preprocess_text(consulta['QueryText'])
            writer.writerow({'QueryNumber': consulta['QueryNumber'], 'QueryText': processed_text})
    logging.info("Arquivo de consultas processadas gerado com sucesso.")

#Função para gerar arquivos csvs dos resultados esperados
def generate_expected_results(consultas, path):
    logging.info(f"Arquivo de resultados esperados começou a ser criado no caminho: {path}.")
    with open(path, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['QueryNumber', 'DocNumber', 'DocVotes']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')
        writer.writeheader()
        for consulta in consultas:
            query_number = consulta['QueryNumber']
            for record in consulta['Records']:
                '''
                  Exemplo de saída:
                  RECORDS
                  {'DocNumber': '159', 'Score': '1002'}

                  CHARACTERS IN record['Score']
                  ['1', '0', '0', '2']
                '''
                votes = sum([1 for char in record['Score'] if char != '0'])
                writer.writerow({'QueryNumber': query_number, 'DocNumber': record['DocNumber'], 'DocVotes': votes})
    logging.info("Arquivo de resultados esperados gerado com sucesso.")




In [6]:
#Criando o arquivo de configuração
create_pc_cfg(path)
#Lendo o arquivo de configuração
cfg = read_config(os.path.join(path, 'PC.CFG'))

#Leitura do arquivo cfquery.xml, contendo as consultas
consultas = processar_xml(os.path.join(path, cfg['LEIA']))
generate_processed_queries(consultas, os.path.join(path, cfg['CONSULTAS']))
generate_expected_results(consultas, os.path.join(path, cfg['ESPERADOS']))

In [7]:
processed_queries = pd.read_csv('/content/data/processed_queries.csv', sep = ";")
expected_results = pd.read_csv('/content/data/expected_results.csv', sep = ";")

In [8]:
processed_queries.head(2)

,QueryNumber,QueryText
0,1,EFFECT CALCIUM PHYSICAL PROPERTY MUCUS CF PATIENT
1,2,ONE DISTINGUISH EFFECT MUCUS HYPERSECRETION INFECTION SUBMUCOSAL GLAND RESPIRATORY TRACT CF


In [9]:
expected_results.head(2)

,QueryNumber,DocNumber,DocVotes
0,1,139,4
1,1,151,4


# **Gerando a lista invertida**
A função desse módulo é criar as listas invertidas simples. A maneira que a criação de listas invertidas funciona é a seguinte: para cada RECORDNUM, coletamos um texto que fica dentro da tag ABSTRACT ou EXTRACT. Depois dos textos coletados e passados pela função de processamento de texto, para cada linha, teremos uma coluna com uma palavra e uma outra coluna que representa uma lista de números que mostra onde essa palavra apareceu e quantas vezes ela apareceu.
* Exemplo: PSEUDOMONAS	[1, 1, 1, 7, 18, 18]

A lista acima indica que a palavra PSEUDOMONAS apareceu no documento 1 por 3 vezes, no documento 7 por 1 vez e no documento 18 por 2 vezes.


In [10]:
#Primeiro irei criar a função de criar e ler o novo arquivo de configuração para geração de listas invertidas
def create_gli_cfg(path):
    logging.info(f"Arquivo GLI.CFG começou a ser criado")
    gli_cfg_content = """LEIA=cf74.xml
    LEIA=cf75.xml
    LEIA=cf76.xml
    LEIA=cf77.xml
    LEIA=cf78.xml
    LEIA=cf79.xml
    ESCREVA=inverted_list.csv"""

    gli_cfg_path = os.path.join(path, 'GLI.CFG')
    with open(gli_cfg_path, 'w') as cfg_file:
        cfg_file.write(gli_cfg_content)
    logging.info(f"Arquivo GLI.CFG criado com sucesso")

def read_gli_config(path):
    logging.info("Lendo arquivo de configuração")
    config = {'LEIA': [], 'ESCREVA': None}
    with open(path, 'r') as file:
        for line in file:
            key, value = line.strip().split('=')
            if key == 'LEIA':
                config['LEIA'].append(value)
            elif key == 'ESCREVA':
                config['ESCREVA'] = value
    return config

In [11]:
from collections import defaultdict

#Cria o pré processamento para lista invertida e extrai os valores das TAGs solicitadas
def recover_and_process_token_in_abstract(path):
    word_to_docs = defaultdict(list)
    for file_path in path:
        logging.info(f"Processando arquivo XML: {file_path}")
        tree = ET.parse(file_path)
        root = tree.getroot()
        for record in root.findall('.//RECORD'):
            record_num = record.find('RECORDNUM').text
            abstract_text = record.find('ABSTRACT').text if record.find('ABSTRACT') is not None else record.find('EXTRACT').text if record.find('EXTRACT') is not None else ""

            for word in preprocess_text(abstract_text).split():
                logging.info(f"Usa a função de processamento de texto no módulo anterior para a palavra {word}.")
                word_to_docs[word].append(record_num)
    logging.info("Processamento de arquivos XML concluído.")
    return word_to_docs

#Gera o arquivo CSV da lista invertida
def generate_inverted_list_csv(word_to_docs, path):
    logging.info(f"Começou a ser criado o csv da lista invertida")
    with open(path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, delimiter=';')
        for word, docs in word_to_docs.items():
            #Letras maiúsculas somente
            word_clean = word.upper()
            '''
              Converte os IDs de documentos em inteiros para remover zeros à esquerda
              e converte a lista de inteiros de volta em uma string que representa a lista de Python.
              Isso foi criado para retirar os zeros a esquerda, fazendo com o que o resultado final
              fique mais parecido com o exemplo que o professor mostrou.
            '''
            docs_int = [int(doc) for doc in docs]
            docs_str = '[' + ', '.join(map(str, docs_int)) + ']'
            writer.writerow([word_clean, docs_str])
    logging.info(f"Lista invertida gerada com sucesso em: {path}")

#Encapsula todos os comandos para a geração da lista invertida.
def generate_inverted_list_from_config(path):
    logging.info("Iniciando geração da lista invertida a partir da configuração.")
    create_gli_cfg(path)
    cfg_path = os.path.join(path, 'GLI.CFG')
    gli_config = read_gli_config(cfg_path)
    file_paths = [os.path.join(path, file_name) for file_name in gli_config['LEIA']]
    word_to_docs = recover_and_process_token_in_abstract(file_paths)
    output_path = os.path.join(path, gli_config['ESCREVA'])
    generate_inverted_list_csv(word_to_docs, output_path)
    logging.info("Geração da lista invertida concluída.")


In [12]:
generate_inverted_list_from_config(path)

In [13]:
inverted_list = pd.read_csv('/content/data/inverted_list.csv', sep = ";")

In [14]:
inverted_list.head(2)

,SIGNIFICANCE,"[1, 74, 78, 121, 147, 157, 179, 185, 195, 205, 223, 258, 319, 355, 402, 411, 412, 485, 526, 555, 593, 631, 809, 858, 923, 1151]"
0,PSEUDOMONAS,"[1, 1, 1, 7, 8, 18, 18, 61, 61, 62, 62, 62, 62, 62, 79, 80, 81, 81, 81, 81, 82, 82, 82, 123, 123, 123, 135, 148, 152, 159, 160, 161, 176, 177, 177, 178, 188, 188, 188, 188, 200, 200, 250, 260, 265, 271, 282, 282, 384, 384, 394, 427, 427, 447, 458, 459, 460, 469, 479, 505, 550, 550, 556, 556, 590, 590, 611, 668, 696, 753, 778, 778, 778, 778, 778, 778, 778, 800, 800, 800, 802, 842, 865, 884, 890, 890, 905, 905, 905, 905, 905, 905, 912, 912, 913, 922, 987, 989, 1000, 1010, 1042, 1042, 1065, 1065, 1065, 1065, 1071, 1071, 1071, 1077, 1077, 1086, 1090, 1091, 1091, 1091, 1112, 1171, 1180, 1192, 1203, 1203, 1227]"
1,AERUGINOSA,"[1, 1, 1, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 18, 18, 62, 79, 80, 80, 81, 110, 123, 123, 135, 148, 159, 159, 159, 160, 161, 161, 161, 176, 177, 177, 178, 178, 178, 178, 178, 200, 250, 260, 260, 260, 260, 265, 265, 271, 282, 346, 346, 394, 447, 447, 447, 447, 447, 451, 451, 451, 458, 458, 458, 459, 460, 460, 460, 460, 469, 469, 469, 469, 469, 469, 469, 469, 479, 479, 505, 550, 588, 588, 588, 588, 588, 588, 588, 588, 588, 589, 589, 589, 589, 589, 589, 590, 590, 591, 591, 611, 668, 668, 696, 753, 753, 753, 778, 778, 778, 778, 778, 778, 778, 790, 790, 790, 790, 790, 800, 800, 802, 842, 874, 884, 884, 884, 890, 890, 890, 890, 890, 905, 912, 912, 912, 912, 912, 912, 912, 917, 917, 917, 922, 922, 922, 945, 945, 983, 983, 983, 983, 983, 987, 987, 987, 988, 988, 988, 989, 989, 989, 989, 989, 1000, 1065, 1071, 1077, 1078, 1086, 1086, 1086, 1086, 1091, 1091, 1112, 1112, 1112, 1171, 1171, 1171, 1173, 1173, 1173, 1173, 1203, 1227]"


# **Indexador**
A função desse módulo é criar o modelo vetorial, dadas as listas invertidas simples. Para isso é necessário calcular o TF-IDF das palavras dos documentos.

* **TF:** Term Frequency, representa a frequência com que as palavras
aparecem em um documento

* **IDF**: É a frequência inversa do documento. Ou seja, é uma métrica que identifica o quão importante é uma palavra. Importante nesse contexto, significa uma palavra que apareça muitas vezes em um documento, mas poucas vezes em outros documentos.

No final, ambos os termos são multiplicados.

**Obs.:** Já existe uma biblioteca em Python que realiza todo esse cálculo chamada ***TfidfVectorizer***

In [15]:
#Cria um arquivo de configuração para o módulo Indexador
def create_index_cfg(path):
    logging.info("Criando arquivo de configuração INDEX.CFG")
    index_cfg_content = """LEIA=inverted_list.csv
    ESCREVA=vector_model.csv"""

    index_cfg_path = os.path.join(path, 'INDEX.CFG')
    with open(index_cfg_path, 'w') as cfg_file:
        cfg_file.write(index_cfg_content)
    logging.info("INDEX.CFG  criado com sucesso.")

#Lê o arquivo de configuração e retorna um dicionário com as configurações
def read_index_config(path):
    logging.info("Lendo arquivo de configuração")
    config = {}
    with open(path, 'r') as file:
        for line in file:
            key, value = line.strip().split('=')
            config[key.strip()] = value.strip()
    return config

In [16]:
import math
from collections import defaultdict
'''
Converte a palavra para maiúscula e verifica se é válida,
Uma palavra válida deve ter tamanho maior ou igual a 2
'''
def preprocess_word(word):
    word = word.upper()
    return word if word.isalpha() and len(word) >= 2 else None

#Carrega a lista invertida e calcula o TF-IDF usando a biblioteca TfidfVectorizer
def calculate_tf_idf(inverted_list_path):

    #Dicionário para armazenar a frequência de cada palavra em cada documento individualmente
    word_documents = defaultdict(lambda: defaultdict(int))
    '''
     Dicionário para armazenar a frequência de documentos para cada palavra.
     Ou seja, em quantos documentos a palavra aparece
     Usar um conjunto (set) automaticamente garante que cada doc_id seja único para cada palavra,
     essencial para o cálculo posterior do IDF (Inverse Document Frequency).
    '''
    document_frequency = defaultdict(set)

    #Abre o arquivo CSV que contém a lista invertida de documentos e palavras
    with open(inverted_list_path, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=';')
        for row in reader:
            #Cada linha tem uma palavra e uma lista de IDs de documentos entre colchetes
            word, doc_ids_str = row
            '''
              LINHA DA LISTA INVERTIDA (exemplo)
                word          count
              ['COUNTING', '[708, 709, 1142, 1142, 1230]']
            '''

            processed_word = preprocess_word(word)
            if processed_word:
                #Removendo colchetes e espaços, e convertendo para inteiros
                doc_ids = [int(doc_id) for doc_id in doc_ids_str.strip('[]').split(',') if doc_id.strip()]
                for doc_id in doc_ids:
                    for doc_id in doc_ids:
                      word_documents[doc_id][processed_word] += 1
                      document_frequency[processed_word].add(doc_id)

    #Calcula TF (Term Frequency)
    N = len(word_documents)
    idf = {word: math.log(N / len(docs)) for word, docs in document_frequency.items()}

    #Calcula IDF (Inverse Document Frequency)
    tf_idf_matrix = {}
    for doc_id, words in word_documents.items():
        tf_idf_matrix[doc_id] = {}
        for word, count in words.items():
            '''
            words.items() exemplo
            dict_items([('LEVEL', 336), ('NORMAL', 717), ('METHOD', 193), ('USED', 141), ('ALSO', 232) ...])
            '''

            tf = count / sum(words.values())
            logging.info("Calculando TF-IDF")
            tf_idf_matrix[doc_id][word] = tf * idf[word]

    #Retorna a matriz de relevância, os nomes das palavras e as ids dos documentos
    return tf_idf_matrix, list(document_frequency.keys()), list(word_documents.keys())

#Salva o modelo vetorial TF-IDF em um arquivo CSV.
def save_vector_model_to_csv(tfidf_matrix, feature_names, doc_ids, path):
    # Cria o DataFrame a partir do dicionário tf_idf_matrix
    df = pd.DataFrame.from_dict(tfidf_matrix, orient='index').fillna(0)

    # Reordena as colunas para garantir que estão em ordem alfabética
    # exceto a primeira coluna que é o 'DocID'
    feature_names_sorted = sorted(feature_names)
    df = df.reindex(columns=feature_names_sorted, fill_value=0)

    # Define 'DocID' como o nome do índice do DataFrame
    df.index.name = 'DocID'

    # Salva o DataFrame em um arquivo CSV usando ';' como separador
    df.to_csv(path, sep=';')
    logging.info(f"Modelo vetorial salvo em: {path}")

'''
Processa a indexação conforme especificado no arquivo de configuração
e encapsula a geração do modelo vetorial.
'''
def generate_vector_model_from_config(cfg_path, path):
    logging.info("Começando o processo de indexação.")
    config = read_index_config(cfg_path)
    inverted_list_path = os.path.join(path, config['LEIA'])
    vector_model_path = os.path.join(path, config['ESCREVA'])


    tfidf_matrix, feature_names, doc_ids = calculate_tf_idf(inverted_list_path)
    logging.info("Cálculo de relevância pelo processo TF-IDF concluído.")
    save_vector_model_to_csv(tfidf_matrix, feature_names, doc_ids, vector_model_path)
    logging.info("Arquivo CSV do modelo vetorial salvo com sucesso.")
    logging.info("Processo de indexação completada com sucesso.")

In [17]:
#Cria o arquivo de configuração INDEX.CFG
create_index_cfg(path)

#Define o caminho para o arquivo de configuração
cfg_path = os.path.join(path, 'INDEX.CFG')

#Executa o processo de indexação
generate_vector_model_from_config(cfg_path, path)

In [18]:
modelo_vetorial = pd.read_csv('/content/data/vector_model.csv', sep = ";")


In [19]:
modelo_vetorial.head(2)

,DocID,AA,AB,ABANDON,ABANDONED,ABDOMEN,ABDOMINAL,ABERRANT,ABERRATION,ABETALIPOPROTINAEMIA,ABH,ABILITY,ABLE,ABNORMAL,ABNORMALITY,ABNORMALLY,ABO,ABOLISHED,ABOLITION,ABORTION,ABPA,ABRADING,ABRASION,ABROAD,ABSCESS,ABSENCE,ABSENT,ABSOLUTE,ABSOLUTELY,ABSORB,ABSORBED,ABSORPTIOMETRY,ABSORPTION,ABSORPTIONIE,ABSORPTIVE,ABSTRACT,ABUNDANT,ABUSE,ACADEMICIAN,ACCELERATED,ACCELERATES,ACCEPT,ACCEPTABLE,ACCEPTANCE,ACCEPTED,ACCEPTING,ACCEPTOR,ACCESS,ACCESSIBLE,ACCIDENTAL,ACCIDENTALLY,ACCOMPANIED,ACCOMPLISHED,ACCOMPLISHMENT,ACCORD,ACCORDANCE,ACCORDED,ACCORDING,ACCORDINGLY,ACCOUNT,ACCOUNTED,ACCOUNTING,ACCUMULATED,ACCUMULATING,ACCUMULATION,ACCURACY,ACCURATE,ACCURATELY,ACETAMINOPHEN,ACETATE,ACETIC,ACETYL,ACETYLATED,ACETYLATION,ACETYLCHOLINE,ACETYLCHOLINEINDUCED,ACETYLCHOLINESTERASE,ACETYLCYSTEINE,ACETYLGLUCOSAMINIDASE,ACETYLSALICYLATE,ACHIEVE,ACHIEVED,ACHIEVEMENT,ACHROMATOPSIA,ACHYLIA,ACID,ACIDAEMIAS,ACIDAND,ACIDBASE,ACIDCONTAINING,ACIDGLUCOSE,ACIDHYDROLASE,ACIDHYDROLASES,ACIDIC,ACIDIFICATION,ACIDITY,ACIDOPATHIES,ACIDOSIS,ACIDSENSITIVE,ACIDSOLUBLE,ACIDTHERMOSTABLE,ACIDTREATED,ACINAR,ACINUS,ACKNOWLEDGED,ACQUIRE,ACQUIRED,ACQUIRING,ACQUISITION,ACROSS,ACRYLAMIDE,ACT,ACTED,ACTIN,ACTING,ACTINSPECIFIC,ACTION,ACTIVATED,ACTIVATING,ACTIVATION,ACTIVATOR,ACTIVE,ACTIVELY,ACTIVITES,ACTIVITY,ACTUAL,ACTUALLY,ACUITY,ACUTE,ACUTELY,ACUTEPHASE,ACYLTRANSFERASE,ADA,ADAPT,ADAPTATION,ADAPTED,ADAPTING,ADD,ADDED,ADDING,ADDITION,ADDITIONAL,ADDITIONALLY,ADDITIVE,ADDRESS,ADDRESSED,ADELAIDE,ADENOIDAL,ADENOMA,ADENOMATOUS,ADENOSINE,ADENOSINETRIPHOSPHATASE,ADENYLATE,ADEQUACY,ADEQUATE,ADEQUATELY,ADH,ADHERED,ADHERENCE,ADJACENT,ADJUNCT,ADJUSTED,ADJUSTING,ADJUSTMENT,ADJUVANT,ADJUVANTICITY,ADMINISTER,ADMINISTERED,ADMINISTRATION,ADMINISTRATIVE,ADMINISTRATOR,ADMISSION,ADMITTED,ADMIXTURE,ADOLESCENCE,ADOLESCENT,ADOPT,ADOPTED,ADP,ADPINDUCED,ADRENAL,ADRENERGIC,ADSORBS,ADSORPTION,ADULT,ADULTHOOD,ADVANCE,ADVANCED,ADVANCING,ADVANTAGE,ADVANTAGEOUS,ADVENT,ADVERSE,ADVERSELY,ADVERTISED,ADVICE,ADVISED,ADVISING,ADVOCATE,AER,AEROBIC,AEROSOL,AEROSOLIZATION,AEROSOLIZED,AERUGINOSA,AETIOLOGICAL,AETIOLOGY,AEURGINOSA,AFFECT,AFFECTED,AFFECTING,AFFECTION,AFFINITY,AFFLICT,AFFLICTED,AFFLICTS,AFFORDED,AFP,AFRICA,AFRICAN,AFRIKANER,AFROAMERICANS,AGAR,AGAROSE,AGE,AGEADJUSTED,AGED,AGEDEPENDENT,AGEGROUP,AGEGROUPS,AGEING,AGELINKED,AGEMATCHED,AGENESIS,AGENT,AGERELATED,AGGLUTINATED,AGGLUTINATES,AGGLUTINATING,AGGLUTINATION,AGGLUTININ,AGGRAVATE,AGGRAVATED,AGGREGATE,AGGREGATED,AGGREGATION,AGGREGOMETER,AGGRESSIVE,AGGRESSIVENESS,AGGRESSIVITY,AGING,AGNESE,AGO,AGONIST,AGREE,AGREED,AGREEMENT,AGREES,AI,AIA,AIB,AIC,AID,AIDED,AIDING,AIM,AIMED,AIP,AIPACTIVITIES,AIPVALUES,AIR,AIREXCHANGE,AIRFLOW,AIRLINE,AIRWAY,AL,ALBEIT,ALBICANS,ALBINISM,ALBUMAID,ALBUMEN,ALBUMIN,ALBUMING,ALBUMINGLOBULIN,ALCIAN,ALCIANOPHILIA,ALCOHOL,ALCOHOLIC,ALCOHOLISM,ALDOSTERONE,ALERT,ALHADEFF,ALIKE,ALIMENTARY,ALIVE,ALKALINE,ALKALOSIS,ALKYLATION,ALLEGATION,ALLEGED,ALLELE,ALLELIC,ALLERGEN,ALLERGIC,ALLERGY,ALLEVIATE,ALLIED,ALLOCATED,ALLOGRAFT,ALLOW,ALLOWANCE,ALLOWED,ALLOWING,ALLOWS,ALMOST,ALONE,ALONG,ALP,ALPHA,ALPHAADRENERGIC,ALPHAAMINO,ALPHAAMYLASE,ALPHAAMYLASES,ALPHADMANNOSIDASE,ALPHAFETOPROTEIN,ALPHAFP,ALPHAFUCOSIDASE,ALPHAGALACTOSIDASE,ALPHAGLUCOSIDASE,ALPHAHAEMOLYSIN,ALPHAHEMOLYSIN,ALPHAL,ALPHALFUCOSIDASE,ALPHALFUCOSIDASES,ALPHAMANNOSIDASE,ALPHAN,ALPHANAPHTHYLPHOSPHATE,ALPHATOCOPHEROL,ALPINE,ALREADY,ALSO,ALSOI,ALTER,ALTERATION,ALTERED,ALTERING,ALTERNATE,ALTERNATED,ALTERNATION,ALTERNATIVE,ALTERNATIVELY,ALTERNATIVEPATHWAY,ALTERS,ALTHOUGH,ALTITUDE,ALTITUDINAL,ALTLAND,ALTOGETHER,ALUMINIUM,ALVEOLAR,ALVEOLARARTERIAL,ALVEOLUS,ALWAYS,AMANTADINE,AMBEIC,AMBULATORY,AMEBOCYTES,AMELIA,AMELIORATE,AMELIORATING,AMENABLE,AMENT,AMERICA,AMERICAN,AMETACHROMATIC,AMICON,AMIDATION,AMIDO,AMIKACIN,AMINE,AMINO,AMINOACIDURIA,AMINOGLYCOSIDE,AMINOGLYCOSIDES,AMINOISOBUTYRATE,AMINOPENICILLIN,AMINOPTERIN,AMMONIUM,AMMONIUMSULPHATEPRECIPITATED,AMNIOCENTESIS,AMNIOTIC,AMNIOTICFLUID,AMONG,AMONGST,AMORPHOUS,AMOUNG,AMOUNT,AMOUNTED,AMOUNTING,AMP,AMPICILLIN,AMPLE,AMPOLA,AMY,AMYALASE,AMYLASE,AMYLOID,AMY

In [20]:
modelo_vetorial.shape

(1215, 9029)

# **Buscador**

O módulo de buscas visa recuperar informações relevantes de um conjunto pré-processado de dados chamado modelo vetorial. Eis o seu funcionamento:

* **Leitura das Consultas Processadas:**

  O módulo lê um arquivo de consultas processadas, onde cada linha contém uma consulta representada pela coluna "QueryText".
  
* **Verificação no Modelo Vetorial:**
  
  Para cada consulta, os termos listados em "QueryText" são comparados com os dados no modelo vetorial. O modelo vetorial é uma matriz onde cada célula contém o valor do TF-IDF, que mede a relevância das palavras nos documentos. Cada linha representa um documento diferente e cada coluna representa um termo diferente.

* **Geração de Resultados:**

  O módulo calcula a similaridade de cosseno entre a consulta e cada documento no modelo vetorial. Esta medida indica o quão relevante é cada documento em relação à consulta.

* **Output de Resultados:**

  Os resultados são organizados em um dataframe chamado resultado_consultas. Cada linha deste dataframe corresponde a uma consulta e contém duas colunas:
    * **search_ID**: Identifica a consulta, correspondendo ao "QueryNumber" da consulta processada original.
    
    * **ranking, doc number, cosine distance:** Apresenta uma lista ordenada de documentos baseada na similaridade. Cada elemento da lista é uma tupla que inclui:
      
        1.   Posição do documento no ranking de relevância.
        2.   Número identificador do documento.
        3.   Valor de similaridade de cosseno, indicando a distância do documento para a consulta.




In [21]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

#Cria o arquivo de configuração para o buscador
def create_busca_cfg(main_path):
    logging.info("Criando arquivo de configuração BUSCA.CFG")
    busca_cfg_content = """MODELO=vector_model.csv
    CONSULTAS=processed_queries.csv
    RESULTADOS=RESULTADOS.csv"""
    busca_cfg_path = os.path.join(main_path, 'BUSCA.CFG')
    with open(busca_cfg_path, 'w') as cfg_file:
        cfg_file.write(busca_cfg_content)
    logging.info("BUSCA.CFG criado com sucesso.")

#Lê o arquivo de configuração para o buscador
def read_busca_config(cfg_path):
    logging.info("Lendo arquivo de configuração BUSCA.CFG")
    config = {}
    with open(cfg_path, 'r') as file:
        for line in file:
            key, value = line.strip().split('=')
            config[key] = value
    return config

In [22]:
'''
Essa função é necessária por rankear as palavras de busca através do método de similaridade
dos cossenos.
'''
def search_and_rank(model_df, query_terms):
    logging.info("Inicializa um vetor de buscas com zeros.")
    #Inicia um vetor com zeros, com a mesma quantidade de colunas que o modelo vetorial
    query_vec = np.zeros((1, model_df.shape[1]))
    '''
      Para cada termo da consulta processada, vamos verificar se o mesmo está presente no
      modelo vetorial. Se ele está presente, a posição correspondente a esse termo no vetor
      de consulta recebe o valor 1, caso contrário, permanece 0.
    '''
    for term in query_terms:
        if term in model_df.columns:
            #Atribui o valor 1 caso o termo esteja presente no modelo vetorial
            query_vec[0, model_df.columns.get_loc(term)] = 1

    '''
    A similaridade entre duas palavras será dado pelo método dos Cossenos
    O método flatten é usado para converter essa matriz de uma linha em um vetor.
    Cada elemento desse vetor contém o valor de similaridade de cosseno entre a consulta e
    um respectivo documento no modelo.
    '''
    similarity = cosine_similarity(query_vec, model_df).flatten()

    #Ordena com base na similaridade do cosseno, do mais similar para o menos similar.
    #O enumerate serve apenas para não perder a referência do documento.
    ranked_results = sorted(enumerate(similarity, start=1), key=lambda x: x[1], reverse=True)

    '''
    A lista de tuplas com os identificadores dos documentos e seus respectivos scores
    de similaridade é retornada como resultado final dessa função.
    '''
    return [(model_df.index[doc_index-1], score) for doc_index, score in ranked_results]

#Escreve os resultados dos termos da busca em um csv
def write_search_results_to_csv(results, output_path):
    with open(output_path, mode='w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, delimiter=';')
        #Escreve o cabeçalho do arquivo CSV
        writer.writerow(['search_ID', 'ranking, doc number, cosine distance'])

        for query_number, doc_scores in results.items():
            #Prepara uma string que representa uma lista de tuplas
            results_str = ', '.join(f'({rank}, {doc_id}, {score:.2f})' for rank, (doc_id, score) in enumerate(doc_scores, start=1))
            #Escreve a linha no CSV como uma representação de uma lista de tuplas
            writer.writerow([query_number, f"[{results_str}]"])

#Realiza a busca de fato
def perform_search(model_path, queries_path, results_path):
    logging.info("Lendo o csv do modelo vetorial.")
    model_df = pd.read_csv(model_path, index_col='DocID', sep=';')

    logging.info("Lendo o csv de consultas processadas.")
    queries_df = pd.read_csv(queries_path, sep=';')

    #Dicionário para armazenar o resultado das buscas
    results = {}

    for index, row in queries_df.iterrows():
        query_terms = row['QueryText'].upper().split()
        ranked_results = search_and_rank(model_df, query_terms)
        results[row['QueryNumber']] = ranked_results

    logging.info("Escreve os resultados da busca para um arquivo CSV.")
    write_search_results_to_csv(results, results_path)


def run_buscador(cfg_path, path):
    logging.info("Começando o processo de busca.")
    config = read_busca_config(cfg_path)
    model_path = os.path.join(path, config['MODELO'])
    queries_path = os.path.join(path, config['CONSULTAS'])
    results_path = os.path.join(path, config['RESULTADOS'])
    perform_search(model_path, queries_path, results_path)
    logging.info("Processo de busca completado com sucesso.")

In [23]:
#Cria o arquivo de configuração BUSCA.CFG
create_busca_cfg(path)

#Define o caminho para o arquivo de configuração
cfg_path = os.path.join(path, 'BUSCA.CFG')

#Executa o processo de busca
run_buscador(cfg_path, path)

In [24]:
consultas_processadas = pd.read_csv("/content/data/processed_queries.csv", sep = ";")
resultado_consultas = pd.read_csv("/content/data/RESULTADOS.csv", sep = ";")

Eis o funcionamento desse sistema de busca:



*   Verifique o search_ID do dataframe contendo os resultados das buscas
*   Verifique a coluna "ranking, doc number, cosine distance" do dataframe contendo os resultados das buscas.
*    Essa coluna apresenta uma lista de tuplas em ordem de relevância ( pelo método dos cossenos)
*   Exemplo: Seja a primeira tupla dessa coluna, a tupla (1, 962, 0.55), cujo Search_ID é igual a 1. Essa tupla diz que o documento ("QueryText") do csv de consultas processadas cujo filtro é QueryNumber == Search_ID == 1, tem uma similaridade de 55% com o DocID == 962 do documento de modelo vetorial ao pesquisar pelas mesmas palavras contidas em QueryText, que no caso é "EFFECT CALCIUM PHYSICAL PROPERTY MUCUS CF PATIENT".


In [25]:
consultas_processadas.head()

,QueryNumber,QueryText
0,1,EFFECT CALCIUM PHYSICAL PROPERTY MUCUS CF PATIENT
1,2,ONE DISTINGUISH EFFECT MUCUS HYPERSECRETION INFECTION SUBMUCOSAL GLAND RESPIRATORY TRACT CF
2,3,SALIVARY GLYCOPROTEIN CF PATIENT DIFFERENT NORMAL SUBJECT
3,4,LIPID COMPOSITION CF RESPIRATORY SECRETION
4,5,CF MUCUS ABNORMAL



**Exemplo do resultado das consultas:**

"search_ID";              "ranking, doc number, cosine distance"

---


   1;                   [(1, 962, 0.55), (2, 969, 0.53), (3, 543, 0.53), (4, 11, 0.52)...]

In [26]:
modelo_vetorial[modelo_vetorial['DocID'] == 962].head(1)

,DocID,AA,AB,ABANDON,ABANDONED,ABDOMEN,ABDOMINAL,ABERRANT,ABERRATION,ABETALIPOPROTINAEMIA,ABH,ABILITY,ABLE,ABNORMAL,ABNORMALITY,ABNORMALLY,ABO,ABOLISHED,ABOLITION,ABORTION,ABPA,ABRADING,ABRASION,ABROAD,ABSCESS,ABSENCE,ABSENT,ABSOLUTE,ABSOLUTELY,ABSORB,ABSORBED,ABSORPTIOMETRY,ABSORPTION,ABSORPTIONIE,ABSORPTIVE,ABSTRACT,ABUNDANT,ABUSE,ACADEMICIAN,ACCELERATED,ACCELERATES,ACCEPT,ACCEPTABLE,ACCEPTANCE,ACCEPTED,ACCEPTING,ACCEPTOR,ACCESS,ACCESSIBLE,ACCIDENTAL,ACCIDENTALLY,ACCOMPANIED,ACCOMPLISHED,ACCOMPLISHMENT,ACCORD,ACCORDANCE,ACCORDED,ACCORDING,ACCORDINGLY,ACCOUNT,ACCOUNTED,ACCOUNTING,ACCUMULATED,ACCUMULATING,ACCUMULATION,ACCURACY,ACCURATE,ACCURATELY,ACETAMINOPHEN,ACETATE,ACETIC,ACETYL,ACETYLATED,ACETYLATION,ACETYLCHOLINE,ACETYLCHOLINEINDUCED,ACETYLCHOLINESTERASE,ACETYLCYSTEINE,ACETYLGLUCOSAMINIDASE,ACETYLSALICYLATE,ACHIEVE,ACHIEVED,ACHIEVEMENT,ACHROMATOPSIA,ACHYLIA,ACID,ACIDAEMIAS,ACIDAND,ACIDBASE,ACIDCONTAINING,ACIDGLUCOSE,ACIDHYDROLASE,ACIDHYDROLASES,ACIDIC,ACIDIFICATION,ACIDITY,ACIDOPATHIES,ACIDOSIS,ACIDSENSITIVE,ACIDSOLUBLE,ACIDTHERMOSTABLE,ACIDTREATED,ACINAR,ACINUS,ACKNOWLEDGED,ACQUIRE,ACQUIRED,ACQUIRING,ACQUISITION,ACROSS,ACRYLAMIDE,ACT,ACTED,ACTIN,ACTING,ACTINSPECIFIC,ACTION,ACTIVATED,ACTIVATING,ACTIVATION,ACTIVATOR,ACTIVE,ACTIVELY,ACTIVITES,ACTIVITY,ACTUAL,ACTUALLY,ACUITY,ACUTE,ACUTELY,ACUTEPHASE,ACYLTRANSFERASE,ADA,ADAPT,ADAPTATION,ADAPTED,ADAPTING,ADD,ADDED,ADDING,ADDITION,ADDITIONAL,ADDITIONALLY,ADDITIVE,ADDRESS,ADDRESSED,ADELAIDE,ADENOIDAL,ADENOMA,ADENOMATOUS,ADENOSINE,ADENOSINETRIPHOSPHATASE,ADENYLATE,ADEQUACY,ADEQUATE,ADEQUATELY,ADH,ADHERED,ADHERENCE,ADJACENT,ADJUNCT,ADJUSTED,ADJUSTING,ADJUSTMENT,ADJUVANT,ADJUVANTICITY,ADMINISTER,ADMINISTERED,ADMINISTRATION,ADMINISTRATIVE,ADMINISTRATOR,ADMISSION,ADMITTED,ADMIXTURE,ADOLESCENCE,ADOLESCENT,ADOPT,ADOPTED,ADP,ADPINDUCED,ADRENAL,ADRENERGIC,ADSORBS,ADSORPTION,ADULT,ADULTHOOD,ADVANCE,ADVANCED,ADVANCING,ADVANTAGE,ADVANTAGEOUS,ADVENT,ADVERSE,ADVERSELY,ADVERTISED,ADVICE,ADVISED,ADVISING,ADVOCATE,AER,AEROBIC,AEROSOL,AEROSOLIZATION,AEROSOLIZED,AERUGINOSA,AETIOLOGICAL,AETIOLOGY,AEURGINOSA,AFFECT,AFFECTED,AFFECTING,AFFECTION,AFFINITY,AFFLICT,AFFLICTED,AFFLICTS,AFFORDED,AFP,AFRICA,AFRICAN,AFRIKANER,AFROAMERICANS,AGAR,AGAROSE,AGE,AGEADJUSTED,AGED,AGEDEPENDENT,AGEGROUP,AGEGROUPS,AGEING,AGELINKED,AGEMATCHED,AGENESIS,AGENT,AGERELATED,AGGLUTINATED,AGGLUTINATES,AGGLUTINATING,AGGLUTINATION,AGGLUTININ,AGGRAVATE,AGGRAVATED,AGGREGATE,AGGREGATED,AGGREGATION,AGGREGOMETER,AGGRESSIVE,AGGRESSIVENESS,AGGRESSIVITY,AGING,AGNESE,AGO,AGONIST,AGREE,AGREED,AGREEMENT,AGREES,AI,AIA,AIB,AIC,AID,AIDED,AIDING,AIM,AIMED,AIP,AIPACTIVITIES,AIPVALUES,AIR,AIREXCHANGE,AIRFLOW,AIRLINE,AIRWAY,AL,ALBEIT,ALBICANS,ALBINISM,ALBUMAID,ALBUMEN,ALBUMIN,ALBUMING,ALBUMINGLOBULIN,ALCIAN,ALCIANOPHILIA,ALCOHOL,ALCOHOLIC,ALCOHOLISM,ALDOSTERONE,ALERT,ALHADEFF,ALIKE,ALIMENTARY,ALIVE,ALKALINE,ALKALOSIS,ALKYLATION,ALLEGATION,ALLEGED,ALLELE,ALLELIC,ALLERGEN,ALLERGIC,ALLERGY,ALLEVIATE,ALLIED,ALLOCATED,ALLOGRAFT,ALLOW,ALLOWANCE,ALLOWED,ALLOWING,ALLOWS,ALMOST,ALONE,ALONG,ALP,ALPHA,ALPHAADRENERGIC,ALPHAAMINO,ALPHAAMYLASE,ALPHAAMYLASES,ALPHADMANNOSIDASE,ALPHAFETOPROTEIN,ALPHAFP,ALPHAFUCOSIDASE,ALPHAGALACTOSIDASE,ALPHAGLUCOSIDASE,ALPHAHAEMOLYSIN,ALPHAHEMOLYSIN,ALPHAL,ALPHALFUCOSIDASE,ALPHALFUCOSIDASES,ALPHAMANNOSIDASE,ALPHAN,ALPHANAPHTHYLPHOSPHATE,ALPHATOCOPHEROL,ALPINE,ALREADY,ALSO,ALSOI,ALTER,ALTERATION,ALTERED,ALTERING,ALTERNATE,ALTERNATED,ALTERNATION,ALTERNATIVE,ALTERNATIVELY,ALTERNATIVEPATHWAY,ALTERS,ALTHOUGH,ALTITUDE,ALTITUDINAL,ALTLAND,ALTOGETHER,ALUMINIUM,ALVEOLAR,ALVEOLARARTERIAL,ALVEOLUS,ALWAYS,AMANTADINE,AMBEIC,AMBULATORY,AMEBOCYTES,AMELIA,AMELIORATE,AMELIORATING,AMENABLE,AMENT,AMERICA,AMERICAN,AMETACHROMATIC,AMICON,AMIDATION,AMIDO,AMIKACIN,AMINE,AMINO,AMINOACIDURIA,AMINOGLYCOSIDE,AMINOGLYCOSIDES,AMINOISOBUTYRATE,AMINOPENICILLIN,AMINOPTERIN,AMMONIUM,AMMONIUMSULPHATEPRECIPITATED,AMNIOCENTESIS,AMNIOTIC,AMNIOTICFLUID,AMONG,AMONGST,AMORPHOUS,AMOUNG,AMOUNT,AMOUNTED,AMOUNTING,AMP,AMPICILLIN,AMPLE,AMPOLA,AMY,AMYALASE,AMYLASE,AMYLOID,AMY

In [27]:
modelo_vetorial[modelo_vetorial['DocID'] == 962].EFFECT

883    0.034725
Name: EFFECT, dtype: float64

In [28]:
modelo_vetorial[modelo_vetorial['DocID'] == 962].CALCIUM

883    0.0
Name: CALCIUM, dtype: float64

In [29]:
modelo_vetorial[modelo_vetorial['DocID'] == 962].PHYSICAL

883    0.0
Name: PHYSICAL, dtype: float64

In [30]:
modelo_vetorial[modelo_vetorial['DocID'] == 962].PROPERTY

883    0.0
Name: PROPERTY, dtype: float64

In [31]:
modelo_vetorial[modelo_vetorial['DocID'] == 962].MUCUS

883    0.0
Name: MUCUS, dtype: float64

In [32]:
modelo_vetorial[modelo_vetorial['DocID'] == 962].CF

883    0.184637
Name: CF, dtype: float64

In [33]:
modelo_vetorial[modelo_vetorial['DocID'] == 962].PATIENT

883    0.221075
Name: PATIENT, dtype: float64

In [34]:
consultas_processadas[(consultas_processadas["QueryNumber"] == 962)]

,QueryNumber,QueryText


# **Criação dos resultados com e sem stemming**

Para fazer o STEMMING, as seguintes instruções devem ser seguidas:
 - Você deve usar o Stemmer de Porter disponível em https://tartarus.org/martin/PorterStemmer/python.txt

  - O script para o stemming de Porter do recurso fornecido pelo professor, precisou ser adaptado para Python 3

 - A palavra STEMMER ou NOSTEMMER deve ser somada a palavra RESULTADOS para formar o nome do arquivo de resposta, resultando em RESULTADOS-STEMMER ou RESULTADOS-NOSTEMMER. Ou seja, 2 csvs precisam ser criados:
RESULTADOS-STEMMER.csv e RESULTADOS-NOSTEMMER.csv e o separador também é o ponto e vírgula





In [35]:
"""
Porter Stemming Algorithm
This is the Porter stemming algorithm, ported to Python from the
version coded up in ANSI C by the author. It may be be regarded
as canonical, in that it follows the algorithm presented in

Porter, 1980, An algorithm for suffix stripping, Program, Vol. 14,
no. 3, pp 130-137,

only differing from it at the points maked --DEPARTURE-- below.

See also http://www.tartarus.org/~martin/PorterStemmer

The algorithm as described in the paper could be exactly replicated
by adjusting the points of DEPARTURE, but this is barely necessary,
because (a) the points of DEPARTURE are definitely improvements, and
(b) no encoding of the Porter stemmer I have seen is anything like
as exact as this version, even with the points of DEPARTURE!

Vivake Gupta (v@nano.com)

Release 1: January 2001

Further adjustments by Santiago Bruno (bananabruno@gmail.com)
to allow word input not restricted to one word per line, leading
to:

release 2: July 2008
"""

import sys

class PorterStemmer:

    def __init__(self):
        """The main part of the stemming algorithm starts here.
        b is a buffer holding a word to be stemmed. The letters are in b[k0],
        b[k0+1] ... ending at b[k]. In fact k0 = 0 in this demo program. k is
        readjusted downwards as the stemming progresses. Zero termination is
        not in fact used in the algorithm.

        Note that only lower case sequences are stemmed. Forcing to lower case
        should be done before stem(...) is called.
        """
        self.b = ""  # buffer for word to be stemmed
        self.k = 0
        self.k0 = 0
        self.j = 0   # j is a general offset into the string

    def cons(self, i):
        """cons(i) is TRUE <=> b[i] is a consonant."""
        if self.b[i] in 'aeiou':
            return False
        if self.b[i] == 'y':
            if i == self.k0:
                return True
            else:
                return not self.cons(i - 1)
        return True

    def m(self):
        """m() measures the number of consonant sequences between k0 and j.
        if c is a consonant sequence and v a vowel sequence, and <..>
        indicates arbitrary presence,

           <c><v>       gives 0
           <c>vc<v>     gives 1
           <c>vcvc<v>   gives 2
           <c>vcvcvc<v> gives 3
           ....
        """
        n = 0
        i = self.k0
        while True:
            if i > self.j:
                return n
            if not self.cons(i):
                break
            i += 1
        i += 1
        while True:
            while True:
                if i > self.j:
                    return n
                if self.cons(i):
                    break
                i += 1
            i += 1
            n += 1
            while True:
                if i > self.j:
                    return n
                if not self.cons(i):
                    break
                i += 1
            i += 1

    def vowelinstem(self):
        """vowelinstem() is TRUE <=> k0,...j contains a vowel"""
        for i in range(self.k0, self.j + 1):
            if not self.cons(i):
                return True
        return False

    def doublec(self, j):
        """doublec(j) is TRUE <=> j,(j-1) contain a double consonant."""
        if j < (self.k0 + 1):
            return False
        if (self.b[j] != self.b[j-1]):
            return False
        return self.cons(j)

    def cvc(self, i):
        """cvc(i) is TRUE <=> i-2,i-1,i has the form consonant - vowel - consonant
        and also if the second c is not w,x or y. this is used when trying to
        restore an e at the end of a short  e.g.

           cav(e), lov(e), hop(e), crim(e), but
           snow, box, tray.
        """
        if i < (self.k0 + 2) or not self.cons(i) or self.cons(i-1) or not self.cons(i-2):
            return False
        ch = self.b[i]
        if ch == 'w' or ch == 'x' or ch == 'y':
            return False
        return True

    def ends(self, s):
        """ends(s) is TRUE <=> k0,...k ends with the string s."""
        length = len(s)
        if s[length - 1] != self.b[self.k]:  # tiny speed-up
            return False
        if length > (self.k - self.k0 + 1):
            return False
        if self.b[self.k-length+1:self.k+1] != s:
            return False
        self.j = self.k - length
        return True

    def setto(self, s):
        """setto(s) sets (j+1),...k to the characters in the string s, readjusting k."""
        length = len(s)
        self.b = self.b[:self.j+1] + s + self.b[self.j+length+1:]
        self.k = self.j + length

    def r(self, s):
        """r(s) is used further down."""
        if self.m() > 0:
            self.setto(s)

    def step1ab(self):
        """step1ab() gets rid of plurals and -ed or -ing. e.g.

           caresses  ->  caress
           ponies    ->  poni
           ties      ->  ti
           caress    ->  caress
           cats      ->  cat

           feed      ->  feed
           agreed    ->  agree
           disabled  ->  disable

           matting   ->  mat
           mating    ->  mate
           meeting   ->  meet
           milling   ->  mill
           messing   ->  mess

           meetings  ->  meet
        """
        if self.b[self.k] == 's':
            if self.ends("sses"):
                self.k -= 2
            elif self.ends("ies"):
                self.setto("i")
            elif self.b[self.k - 1] != 's':
                self.k -= 1
        if self.ends("eed"):
            if self.m() > 0:
                self.k -= 1
        elif (self.ends("ed") or self.ends("ing")) and self.vowelinstem():
            self.k = self.j
            if self.ends("at"):
                self.setto("ate")
            elif self.ends("bl"):
                self.setto("ble")
            elif self.ends("iz"):
                self.setto("ize")
            elif self.doublec(self.k):
                self.k -= 1
                ch = self.b[self.k]
                if ch in ['l', 's', 'z']:
                    self.k += 1
            elif (self.m() == 1 and self.cvc(self.k)):
                self.setto("e")

    def step1c(self):
        """step1c() turns terminal y to i when there is another vowel in the stem."""
        if (self.ends("y") and self.vowelinstem()):
            self.b = self.b[:self.k] + 'i' + self.b[self.k+1:]

    def step2(self):
        """step2() maps double suffices to single ones.
        so -ization ( = -ize plus -ation) maps to -ize etc. note that the
        string before the suffix must give m() > 0.
        """
        if self.b[self.k - 1] == 'a':
            if self.ends("ational"):
                self.r("ate")
            elif self.ends("tional"):
                self.r("tion")
        elif self.b[self.k - 1] == 'c':
            if self.ends("enci"):
                self.r("ence")
            elif self.ends("anci"):
                self.r("ance")
        elif self.b[self.k - 1] == 'e':
            if self.ends("izer"):
                self.r("ize")
        elif self.b[self.k - 1] == 'l':
            if self.ends("bli"):
                self.r("ble")  # --DEPARTURE--
            elif self.ends("alli"):
                self.r("al")
            elif self.ends("entli"):
                self.r("ent")
            elif self.ends("eli"):
                self.r("e")
            elif self.ends("ousli"):
                self.r("ous")
        elif self.b[self.k - 1] == 'o':
            if self.ends("ization"):
                self.r("ize")
            elif self.ends("ation"):
                self.r("ate")
            elif self.ends("ator"):
                self.r("ate")
        elif self.b[self.k - 1] == 's':
            if self.ends("alism"):
                self.r("al")
            elif self.ends("iveness"):
                self.r("ive")
            elif self.ends("fulness"):
                self.r("ful")
            elif self.ends("ousness"):
                self.r("ous")
        elif self.b[self.k - 1] == 't':
            if self.ends("aliti"):
                self.r("al")
            elif self.ends("iviti"):
                self.r("ive")
            elif self.ends("biliti"):
                self.r("ble")
        elif self.b[self.k - 1] == 'g':  # --DEPARTURE--
            if self.ends("logi"):
                self.r("log")
        # To match the published algorithm, delete this phrase

    def step3(self):
        """step3() deals with -ic-, -full, -ness etc. similar strategy to step2."""
        if self.b[self.k] == 'e':
            if self.ends("icate"):
                self.r("ic")
            elif self.ends("ative"):
                self.r("")
            elif self.ends("alize"):
                self.r("al")
        elif self.b[self.k] == 'i':
            if self.ends("iciti"):
                self.r("ic")
        elif self.b[self.k] == 'l':
            if self.ends("ical"):
                self.r("ic")
            elif self.ends("ful"):
                self.r("")
        elif self.b[self.k] == 's':
            if self.ends("ness"):
                self.r("")

    def step4(self):
        """step4() takes off -ant, -ence etc., in context <c>vcvc<v>."""
        if self.b[self.k - 1] == 'a':
            if self.ends("al"):
                pass
            else:
                return
        elif self.b[self.k - 1] == 'c':
            if self.ends("ance"):
                pass
            elif self.ends("ence"):
                pass
            else:
                return
        elif self.b[self.k - 1] == 'e':
            if self.ends("er"):
                pass
            else:
                return
        elif self.b[self.k - 1] == 'i':
            if self.ends("ic"):
                pass
            else:
                return
        elif self.b[self.k - 1] == 'l':
            if self.ends("able"):
                pass
            elif self.ends("ible"):
                pass
            else:
                return
        elif self.b[self.k - 1] == 'n':
            if self.ends("ant"):
                pass
            elif self.ends("ement"):
                pass
            elif self.ends("ment"):
                pass
            elif self.ends("ent"):
                pass
            else:
                return
        elif self.b[self.k - 1] == 'o':
            if self.ends("ion") and (self.b[self.j] == 's' or self.b[self.j] == 't'):
                pass
            elif self.ends("ou"):
                pass
            # takes care of -ous
            else:
                return
        elif self.b[self.k - 1] == 's':
            if self.ends("ism"):
                pass
            else:
                return
        elif self.b[self.k - 1] == 't':
            if self.ends("ate"):
                pass
            elif self.ends("iti"):
                pass
            else:
                return
        elif self.b[self.k - 1] == 'u':
            if self.ends("ous"):
                pass
            else:
                return
        elif self.b[self.k - 1] == 'v':
            if self.ends("ive"):
                pass
            else:
                return
        elif self.b[self.k - 1] == 'z':
            if self.ends("ize"):
                pass
            else:
                return
        else:
            return
        if self.m() > 1:
            self.k = self.j

    def step5(self):
        """step5() removes a final -e if m() > 1, and changes -ll to -l if
        m() > 1.
        """
        self.j = self.k
        if self.b[self.k] == 'e':
            a = self.m()
            if a > 1 or (a == 1 and not self.cvc(self.k-1)):
                self.k -= 1
        if self.b[self.k] == 'l' and self.doublec(self.k) and self.m() > 1:
            self.k -= 1

    def stem(self, p, i, j):
        """In stem(p,i,j), p is a char pointer, and the string to be stemmed
        is from p[i] to p[j] inclusive. Typically i is zero and j is the
        offset to the last character of a string, (p[j+1] == '\0'). The
        stemmer adjusts the characters p[i] ... p[j] and returns the new
        end-point of the string, k. Stemming never increases word length, so
        i <= k <= j. To turn the stemmer into a module, declare 'stem' as
        extern, and delete the remainder of this file.
        """
        self.b = p
        self.k = j
        self.k0 = i
        if self.k <= self.k0 + 1:
            return self.b  # --DEPARTURE--

        self.step1ab()
        self.step1c()
        self.step2()
        self.step3()
        self.step4()
        self.step5()
        return self.b[self.k0:self.k+1]

    def stem_text(self, text):
        """Processa um texto completo aplicando o stemming a cada palavra."""
        output = ''
        word = ''
        for c in text:
            if c.isalpha():
                word += c.lower()
            else:
                if word:
                    output += self.stem(word, 0, len(word) - 1)
                    word = ''
                output += c.lower()
        if word:  # Se ainda resta alguma palavra ao final do texto
            output += self.stem(word, 0, len(word) - 1)
        return output

#EXEMPLO
if __name__ == '__main__':
    # Criando uma instância do stemmer
    e = PorterStemmer()

    # Exemplo de uso:
    text = "Here are some examples of stemming applied to various words."
    stemmed_text = e.stem_text(text)
    print("Original Text:", text)
    print("Stemmed Text:", stemmed_text)


Original Text: Here are some examples of stemming applied to various words.
Stemmed Text: here ar some exampl of stem appli to variou word.


In [36]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import csv
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Caminho para os arquivos
path = '/content/data/'

# Carrega o modelo vetorial
vector_model_df = pd.read_csv(path + 'vector_model.csv', index_col='DocID', sep=';')

# Carrega as consultas processadas
processed_queries_df = pd.read_csv(path + 'processed_queries.csv', sep=';')

# Instanciando a classe PorterStemmer
p = PorterStemmer()

# Função para processar o texto com o algoritmo de Porter para stemming
def preprocess_with_porter(text, porter):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    processed_tokens = [porter.stem(word, 0, len(word) - 1) for word in tokens if word not in stop_words and word.isalpha()]
    return ' '.join(processed_tokens).upper()

# Função para processar o texto sem stemming
def preprocess_without_porter(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    processed_tokens = [word for word in tokens if word not in stop_words and word.isalpha()]
    return ' '.join(processed_tokens).upper()

# Função de busca no modelo vetorial
def search_and_rank(model_df, query_terms):
    query_vec = np.zeros((1, model_df.shape[1]))
    for term in query_terms:
        if term in model_df.columns:
            query_vec[0, model_df.columns.get_loc(term)] = 1
    similarity = cosine_similarity(query_vec, model_df).flatten()
    return [(model_df.index[doc_index - 1], score) for doc_index, score in sorted(enumerate(similarity, start=1), key=lambda x: x[1], reverse=True)]

# Processa consultas com e sem o Porter Stemmer
processed_queries_df['QueryTextStemmed'] = processed_queries_df['QueryText'].apply(lambda x: preprocess_with_porter(x, p))
processed_queries_df['QueryTextNoStem'] = processed_queries_df['QueryText'].apply(preprocess_without_porter)

# Realiza a busca para consultas com e sem stemming
results_stemmed = {}
results_no_stem = {}
for index, row in processed_queries_df.iterrows():
    query_terms_stemmed = row['QueryTextStemmed'].split()
    results_stemmed[row['QueryNumber']] = search_and_rank(vector_model_df, query_terms_stemmed)
    query_terms_no_stem = row['QueryTextNoStem'].split()
    results_no_stem[row['QueryNumber']] = search_and_rank(vector_model_df, query_terms_no_stem)

# Função para escrever os resultados em CSV
def write_results_to_csv(results, output_path):
    with open(output_path, mode='w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, delimiter=';')
        writer.writerow(['search_ID', 'ranking, doc number, cosine distance'])
        for query_number, doc_scores in results.items():
            results_str = ', '.join(f'({rank}, {doc_id}, {score:.2f})' for rank, (doc_id, score) in enumerate(doc_scores, start=1))
            writer.writerow([query_number, f"[{results_str}]"])

# Salva os resultados da busca com e sem stemming
write_results_to_csv(results_stemmed, path + 'RESULTADOS-STEMMER.csv')
write_results_to_csv(results_no_stem, path + 'RESULTADOS-NOSTEMMER.csv')


* O aumento da pontuação de similaridade após o stemming indica que o processo de normalização das palavras (reduzindo-as à sua forma raiz) pode ter ajudado a melhorar a correspondência entre os termos da consulta e os documentos.

* Com o stemming, é provável que variações da mesma palavra (por exemplo, "running", "runs", "ran" reduzidas a "run") sejam consideradas iguais, o que pode levar a uma melhor correspondência com os documentos que contêm qualquer uma dessas variações.

* A diferença nas pontuações de similaridade também pode ser influenciada por como o stemming altera a distribuição de palavras no espaço vetorial do modelo. Isso pode levar a mudanças na forma como os ângulos são calculados entre os vetores de consulta e os vetores de documento, resultando em diferentes pontuações de similaridade.

* Esses resultados podem ser úteis para avaliar se a aplicação do stemming melhora o desempenho do sistema de recuperação de informações em termos de precisão e recall, especialmente se puder compará-los com um conjunto de resultados esperados ou julgamentos de relevância.

# **Classe de plotagem de todas as métricas requisitadas**

1. Gráfico de 11 pontos de precisão e recall
2. F1
3. Precision@5
4. Precision@10
5. Histograma de R-Precision (comparativo)
6. MAP
7. MRR
8. Discounted Cumulative Gain (médio)
9. Normalized Discounted Cumulative Gain



Abaixo está uma breve explicação de todas os cálculos das métricas e como elas funcionam:

* **calculate_precision_recall**:

  A função calculate_precision_recall serve para avaliar a eficácia de um sistema de recuperação de informação ao longo de diferentes níveis de decisão, conhecidos como limiares (thresholds) com valores entre 0 e 1, divididos em 11 pontos igualmente espaçados. O objetivo é determinar quão bem o sistema consegue identificar documentos relevantes entre os resultados retornados.

* **calculate_f1**:

  A função calculate_f1 é usada para avaliar a eficiência geral de um sistema de recuperação de informação, calculando o score F1 para cada ponto na curva de precisão-recall gerada por diferentes limiares de decisão. O valor calculado é a média harmônica entre Precision e Recall.

* **calculate_dcg**:

  A função calculate_dcg é utilizada para medir a eficácia de um sistema de recuperação de informação ao classificar documentos de acordo com sua relevância. A métrica usada aqui é o Ganho Cumulativo Descontado (DCG), que é particularmente útil para situações onde os documentos no topo da lista de resultados são mais importantes do que aqueles mais abaixo.

* **calculate_ndcg**:

  O NDCG serve para avaliar sistemas de busca ou recomendação onde a relevância e a posição dos documentos são muito importantes. Ele fornece uma medida relativa que ajuda a entender não apenas quão bons são os resultados, mas quão próximos eles estão do cenário ideal. Isso permite ajustar seus algoritmos para melhorar a precisão da ordenação dos resultados de pesquisa, otimizando assim a experiência do usuário final.

* **calculate_map**:

  O MAP é uma métrica para avaliar sistemas de recuperação de informação que considera tanto a precisão dos resultados quanto a ordenação dos documentos relevantes. Isso é útil em aplicações onde a ordem dos resultados é crucial, como em sistemas de busca na web ou recomendações personalizadas, pois fornece uma medida compreensiva da qualidade dos resultados retornados ao usuário.

* **calculate_mrr**:

  O MRR é uma métrica utilizada para medir a qualidade dos primeiros resultados retornados por sistemas de busca ou recomendação. Os primeiros resultados precisam ter sua relevância calculada primeiro, em um cenário parecido com a busca da internet.

* **calculate_precision_at_k**:

  A função calculate_precision_at_k foi criada para avaliar a eficiência de sistemas de recuperação de informação ao determinar a proporção de documentos relevantes encontrados entre os primeiros 'k' documentos retornados em uma pesquisa. Esta métrica é conhecida como precisão no rank 'k' e é muito importante para entender a eficácia de um sistema em trazer os k documentos mais relevantes para as posições mais altas nos resultados da pesquisa.

* **calculate_r_precision**:

  A R-Precisão é uma métrica usada em cenários onde o número exato de documentos relevantes é conhecido antecipadamente e varia entre diferentes consultas. Ao focar exatamente no número esperado de documentos relevantes, esta métrica fornece uma avaliação direta da capacidade do sistema de recuperação de identificar todos os documentos relevantes sem excesso ou falta.

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import logging
from zipfile import ZipFile

# Configuração de logging para capturar e exibir informações durante a execução do programa.
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class MetricsAnalyzer:
    def __init__(self, data_path='/content/data/'):
        self.data_path = data_path

    def load_data(self, stem_suffix):
        results_path = os.path.join(self.data_path, f"RESULTADOS-{stem_suffix}.csv")
        expected_path = os.path.join(self.data_path, "expected_results.csv")
        logging.info(f"Carregando dados para o sufixo '{stem_suffix}'...")
        results = pd.read_csv(results_path, delimiter=';', converters={"ranking, doc number, cosine distance": eval})
        expected = pd.read_csv(expected_path, delimiter=';')
        logging.info("Dados carregados com sucesso.")
        return results, expected

    def calculate_precision_recall(self, results, expected):
        '''
          Calcula precisão e recall para vários níveis de limiar.
          results: DataFrame contendo os resultados da pesquisa com pontuações.
          expected: DataFrame contendo os resultados esperados.
          Calcula True Positives (tp), False Positives (fp) e False Negatives (fn) para cada threshold e,
          em seguida, calcula precisão e recall.

          Retorna:
            Tupla de listas de valores de precisão e recall, e os thresholds usados.
        '''
        #Gera um conjunto de thresholds que varia linearmente de 0 a 1, divididos em 11 pontos (incluindo 0 e 1)
        thresholds = np.linspace(0, 1, 11)
        precision = []
        recall = []
        '''
        * Para cada threshold no conjunto gerado, a função itera sobre todos os documentos recuperados para cada consulta.
        * Se a pontuação (score de similaridade dos cossenos) de um documento é maior ou igual ao threshold atual,
        esse documento é considerado como "recuperado" para os propósitos do cálculo de precisão e recall naquele threshold.
        '''
        for threshold in thresholds:
            tp, fp, fn = 0, 0, 0
            for _, row in results.iterrows():
                retrieved = [doc for rank, doc, score in row['ranking, doc number, cosine distance'] if score >= threshold]
                relevant = expected[expected['QueryNumber'] == row['search_ID']]['DocNumber'].tolist()
                tp += len(set(retrieved) & set(relevant))
                fp += len(set(retrieved) - set(relevant))
                fn += len(set(relevant) - set(retrieved))
            precision.append(tp / (tp + fp) if tp + fp > 0 else 0)
            recall.append(tp / (tp + fn) if tp + fn > 0 else 0)
        return precision, recall, thresholds

    def calculate_f1(self, precision, recall):
        '''
          Calcula o score F1 com base nos valores de precisão e recall.
          precision: Lista de valores de precisão.
          recall: Lista de valores de recall.
          F1 é a média harmônica de precisão e recall, calculado para cada ponto na curva de precisão-recall.

          Retorna:
            Lista de scores F1.
        '''
        return [2 * p * r / (p + r) if p + r > 0 else 0 for p, r in zip(precision, recall)]

    def calculate_dcg(self, results, expected):
        '''
          Calcula o Ganho Cumulativo Descontado (DCG).
          results: DataFrame contendo os resultados da pesquisa.
          expected: DataFrame contendo os resultados esperados com as pontuações de relevância.
          DCG é calculado somando os valores de relevância dos documentos recuperados, descontados pelo
          logaritmo da posição em que aparecem.

          Retorna:
            Lista de scores DCG para cada consulta.
        '''
        dcgs = []
        for _, row in results.iterrows():
            scores = [expected.loc[expected['DocNumber'] == doc, 'DocVotes'].values[0] if doc in expected['DocNumber'].values else 0
                      for rank, doc, _ in row['ranking, doc number, cosine distance']][:10]
            dcgs.append(sum(score / np.log2(i + 2) for i, score in enumerate(scores)))
        return dcgs

    def calculate_ndcg(self, dcgs, expected):
        '''
          Calcula o Ganho Cumulativo Descontado Normalizado (NDCG).
          dcgs: Lista de scores DCG.
          expected: DataFrame contendo os resultados esperados com pontuações de relevância.
          NDCG é o DCG normalizado pela divisão pelo DCG ideal, que seria obtido se os documentos estivessem
          na ordem perfeita de relevância.

          Retorna:
            Lista de scores NDCG.
        '''
        ideal_scores = sorted(expected['DocVotes'], reverse=True)[:10]
        ideal_dcg = sum(score / np.log2(i + 2) for i, score in enumerate(ideal_scores))
        return [dcg / ideal_dcg if ideal_dcg > 0 else 0 for dcg in dcgs]

    def calculate_map(self, results, expected):
        '''
          Calcula a Precisão Média da Média (MAP) para cada consulta.
          results: DataFrame contendo os resultados da pesquisa.
          expected: DataFrame contendo os resultados esperados com pontuações de relevância.
          MAP é calculado encontrando a precisão em cada ponto em que um documento relevante é
          recuperado e, em seguida, calculando a média dessas precisões.

          Retorna:
            Média dos scores de Precisão Média.
        '''
        aps = []
        for _, row in results.iterrows():
            relevant = expected[expected['QueryNumber'] == row['search_ID']]['DocNumber'].tolist()
            ap = 0
            hits = 0
            for i, (_, doc, _) in enumerate(row['ranking, doc number, cosine distance']):
                if doc in relevant:
                    hits += 1
                    ap += hits / (i + 1)
            aps.append(ap / len(relevant) if relevant else 0)
        return np.mean(aps)

    def calculate_mrr(self, results, expected):
        '''
          Calcula a Média do Rank Recíproco (MRR).
          results: DataFrame contendo os resultados da pesquisa.
          expected: DataFrame contendo os resultados esperados com pontuações de relevância.
          MRR é calculado encontrando o recíproco do rank do primeiro documento relevante recuperado para
          cada consulta e então calculando a média desses valores.

          Retorna:
            Média dos Ranks Recíprocos.
        '''
        rrs = []
        for _, row in results.iterrows():
            for rank, doc, _ in row['ranking, doc number, cosine distance']:
                if doc in expected[expected['QueryNumber'] == row['search_ID']]['DocNumber'].values:
                    rrs.append(1 / (rank + 1))
                    break
        return np.mean(rrs)

    def calculate_precision_at_k(self, results, expected, k):
        '''
          Calcula a precisão no rank k.
          results: DataFrame contendo os resultados da pesquisa.
          expected: DataFrame contendo os resultados esperados com pontuações de relevância.
          k: Rank no qual a precisão é calculada.
          Precisão em k é basicamente a proporção de documentos relevantes encontrados nos primeiros k documentos retornados.

          Retorna:
            Média da precisão no rank k.
        '''
        precision_at_k = []
        for _, row in results.iterrows():
            relevant_docs = expected[expected['QueryNumber'] == row['search_ID']]['DocNumber'].tolist()
            retrieved_docs = [doc for rank, doc, _ in row['ranking, doc number, cosine distance']][:k]
            tp = len(set(retrieved_docs) & set(relevant_docs))
            precision_at_k.append(tp / k if k else 0)
        return np.mean(precision_at_k)

    def calculate_r_precision(self, results, expected):
        '''
          Calcula a R-Precisão para cada consulta.
          results: DataFrame contendo os resultados da pesquisa.
          expected: DataFrame contendo os resultados esperados com pontuações de relevância.
          R-Precisão é a precisão calculada no ponto onde o número de documentos recuperados
          é igual ao número de documentos relevantes para a consulta.

          Retorna:
            Lista de scores de R-Precisão para cada consulta.
        '''
        r_precision_scores = []
        for _, row in results.iterrows():
            relevant_docs = expected[expected['QueryNumber'] == row['search_ID']]['DocNumber'].tolist()
            r = len(relevant_docs)
            retrieved_docs = [doc for rank, doc, _ in row['ranking, doc number, cosine distance']][:r]
            tp = len(set(retrieved_docs) & set(relevant_docs))
            r_precision_scores.append(tp / r if r else 0)
        return r_precision_scores

    def calculate_metrics(self, results, expected):
        '''
          Função que encapsula todas as funções de cálculos de todas as métricas.
          results: DataFrame contendo os resultados da pesquisa.
          expected: DataFrame contendo os resultados esperados com pontuações de relevância.

          Retorna:
            Dicionário contendo todas as métricas calculadas e os thresholds usados.
        '''
        metrics = {}
        metrics['precision'], metrics['recall'], thresholds = self.calculate_precision_recall(results, expected)
        metrics['f1'] = self.calculate_f1(metrics['precision'], metrics['recall'])
        metrics['precision_at_5'] = self.calculate_precision_at_k(results, expected, 5)
        metrics['precision_at_10'] = self.calculate_precision_at_k(results, expected, 10)
        metrics['r_precision'] = self.calculate_r_precision(results, expected)
        metrics['dcg'] = self.calculate_dcg(results, expected)
        metrics['ndcg'] = self.calculate_ndcg(metrics['dcg'], expected)
        metrics['map'] = self.calculate_map(results, expected)
        metrics['mrr'] = self.calculate_mrr(results, expected)
        return metrics, thresholds

    def save_metrics_to_csv(self, metrics, stem_suffix):
        '''
          Salva as métricas em arquivos CSV.
          metrics: Dicionário contendo todas as métricas calculadas.
          stem_suffix: Sufixo indicando se foi usado stemming ou não ('STEMMER' ou 'NOSTEMMER').
        '''
        for metric, values in metrics.items():
            if isinstance(values, list):
                df = pd.DataFrame(values, columns=[metric])
                df.to_csv(os.path.join(self.data_path, f"{metric}_results_{stem_suffix}.csv"), index=False)
            else:  #Para métricas com valor único
                df = pd.DataFrame({metric: [values]})
                df.to_csv(os.path.join(self.data_path, f"{metric}_results_{stem_suffix}.csv"), index=False)

    def plot_precision_recall_curve(self, precision, recall, thresholds, stem_suffix):
        '''
          Plota e salva o gráfico de 11 pontos de precisão-recall.
          precision: Lista de valores de precisão.
          recall: Lista de valores de recall.
          thresholds: Lista de limites usados para o cálculo de precisão-recall.
          stem_suffix: Sufixo indicando se foi usado stemming ou não ('STEMMER' ou 'NOSTEMMER').
        '''
        plt.figure(figsize=(10, 6))
        plt.plot(recall, precision, marker='o', linestyle='-', label=f'{stem_suffix.capitalize()} Stemming')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title(f'11 Pontos de Precisão e Recall ({stem_suffix.capitalize()} Stemming)')
        plt.grid(True)
        plt.legend()
        plt.savefig(os.path.join(self.data_path, f'11_points_precision_recall_{stem_suffix.lower()}.png'))
        plt.close()

        # Salvar os valores em CSV com separador ponto e vírgula
        df = pd.DataFrame({
            'Recall': recall,
            'Precision': precision
        })
        df.to_csv(os.path.join(self.data_path, f'11_points_precision_recall_{stem_suffix.lower()}.csv'), index=False, sep=';')


    def plot_precision_recall_comparison(self, precision_stem, recall_stem, precision_nostem, recall_nostem):
        '''
          Plota e salva uma comparação das curvas de precisão-recall para dados com e sem stemming.
          precision_stem: Valores de precisão para dados com stemming.
          recall_stem: Valores de recall para dados com stemming.
          precision_nostem: Valores de precisão para dados sem stemming.
          recall_nostem: Valores de recall para dados sem stemming.
        '''
        plt.figure(figsize=(10, 6))
        plt.plot(recall_stem, precision_stem, marker='o', linestyle='-', label='Com Stemming')
        plt.plot(recall_nostem, precision_nostem, marker='o', linestyle='-', label='Sem Stemming')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Comparação dos 11 Pontos de Precisão e Recall')
        plt.grid(True)
        plt.legend()
        plt.savefig(os.path.join(self.data_path, '11_points_precision_recall_comparison.png'))
        plt.close()

    #Plota o gráfico de R-precision
    def plot_r_precision_histogram(self, r_precision_stem, r_precision_nostem):
      '''
      Plota e salva um histograma das diferenças de R-Precisão para dados com e sem stemming.
      r_precision_stem: Valores de R-Precisão para dados com stemming.
      r_precision_nostem: Valores de R-Precisão para dados sem stemming.
      '''
      # Calcular a diferença de R-Precision
      r_precision_differences = np.array(r_precision_stem) - np.array(r_precision_nostem)

      plt.figure(figsize=(10, 6))
      plt.bar(range(1, len(r_precision_differences) + 1), r_precision_differences, color='gray')
      plt.title('Diferença de R-Precisão entre Com Stemming e Sem Stemming')
      plt.xlabel('Número da Consulta')
      plt.ylabel('R-Precision A/B')
      plt.grid(True)
      plt.savefig(os.path.join(self.data_path, 'R-Precision_histogram.png'))
      plt.close()



    #Plota e salva gráficos de barras para uma métrica específica para dados com e sem stemming.
    def plot_metric_barplot(self, metric_values_stem, metric_values_nostem, metric_name):
        plt.figure(figsize=(10, 6))
        bar_width = 0.35
        index = np.arange(1)
        plt.bar(index, metric_values_stem, bar_width, label='With Stemming')
        plt.xlabel('Query ID')
        plt.ylabel(metric_name.capitalize())
        plt.title(f'{metric_name.capitalize()} With Stemming')
        plt.xticks(index, [1])
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(self.data_path, f'{metric_name}_stemming.png'))
        plt.close()

        plt.figure(figsize=(10, 6))
        plt.bar(index, metric_values_nostem, bar_width, label='Without Stemming', color='orange')
        plt.xlabel('Query ID')
        plt.ylabel(metric_name.capitalize())
        plt.title(f'{metric_name.capitalize()} Without Stemming')
        plt.xticks(index, [1])
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(self.data_path, f'{metric_name}_nostemming.png'))
        plt.close()

        plt.figure(figsize=(10, 6))
        plt.bar(index - bar_width/2, metric_values_stem, bar_width, label='With Stemming')
        plt.bar(index + bar_width/2, metric_values_nostem, bar_width, label='Without Stemming', color='orange')
        plt.xlabel('Query ID')
        plt.ylabel(metric_name.capitalize())
        plt.title(f'{metric_name.capitalize()} Comparison')
        plt.xticks(index, [1])
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(self.data_path, f'{metric_name}_comparison.png'))
        plt.close()

    #Plota e salva gráficos de linha para uma métrica específica para dados com e sem stemming
    def plot_metric_lineplot(self, metric_values_stem, metric_values_nostem, metric_name):
        plt.figure(figsize=(10, 6))
        plt.plot(metric_values_stem, label='With Stemming')
        plt.xlabel('Query ID')
        plt.ylabel(metric_name.capitalize())
        plt.title(f'{metric_name.capitalize()} With Stemming')
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(self.data_path, f'{metric_name}_stemming.png'))
        plt.close()

        plt.figure(figsize=(10, 6))
        plt.plot(metric_values_nostem, label='Without Stemming', color='orange')
        plt.xlabel('Query ID')
        plt.ylabel(metric_name.capitalize())
        plt.title(f'{metric_name.capitalize()} Without Stemming')
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(self.data_path, f'{metric_name}_nostemming.png'))
        plt.close()

        plt.figure(figsize=(10, 6))
        plt.plot(metric_values_stem, label='With Stemming')
        plt.plot(metric_values_nostem, label='Without Stemming', color='orange')
        plt.xlabel('Query ID')
        plt.ylabel(metric_name.capitalize())
        plt.title(f'{metric_name.capitalize()} Comparison')
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(self.data_path, f'{metric_name}_comparison.png'))
        plt.close()

    def save_report(self, metrics_stem, metrics_nostem):
        '''
          Salva um resumo textual da análise das métricas.
          metrics_stem: Dicionário de métricas para dados com stemming.
          metrics_nostem: Dicionário de métricas para dados sem stemming.
        '''
        report = "#Resumo da Análise de Métricas\n\n"

        def metric_to_str(metric_name, values):
            if isinstance(values, list):
                values_str = ", ".join(f"{v:.4f}" for v in values)
                return f"**{metric_name}**: {values_str}\n"
            else:
                return f"**{metric_name}**: {values:.4f}\n"

        report += "## Dados com Stemming\n\n"
        for metric_name, values in metrics_stem.items():
            report += metric_to_str(metric_name, values)

        report += "\n## Dados sem Stemming\n\n"
        for metric_name, values in metrics_nostem.items():
            report += metric_to_str(metric_name, values)

        with open(os.path.join(self.data_path, 'RELATORIO.MD'), 'w') as f:
            f.write(report)

    def run(self):
        '''
          Método principal para executar o processo de análise de métricas e geração de csvs e jpgs.
          Abaixo serão executados alguns métodos da biblioteca time para calcularmos o tempo de execução de cada evento
        '''
        start_time = time.time()

        logging.info("Calculando métricas para dados com radicalização...")
        metrics_stem, thresholds = self.calculate_metrics(*self.load_data('STEMMER'))
        logging.info("Calculando métricas para dados sem radicalização...")
        metrics_nostem, _ = self.calculate_metrics(*self.load_data('NOSTEMMER'))

        logging.info("Salvando métricas em arquivos CSV...")
        self.save_metrics_to_csv(metrics_stem, 'STEMMER')
        self.save_metrics_to_csv(metrics_nostem, 'NOSTEMMER')

        logging.info("Plotando curvas de precisão-recall...")
        self.plot_precision_recall_curve(metrics_stem['precision'], metrics_stem['recall'], thresholds, 'STEMMER')
        self.plot_precision_recall_curve(metrics_nostem['precision'], metrics_nostem['recall'], thresholds, 'NOSTEMMER')

        logging.info("Plotando comparação de precisão-recall...")
        self.plot_precision_recall_comparison(metrics_stem['precision'], metrics_stem['recall'], metrics_nostem['precision'], metrics_nostem['recall'])

        logging.info("Plotando gráficos de linha das métricas...")
        for metric in ['f1', 'ndcg', 'dcg']:
            self.plot_metric_lineplot(metrics_stem[metric], metrics_nostem[metric], metric)

        logging.info("Plotando gráficos de barras das métricas...")
        for metric in ['map', 'mrr', 'precision_at_5', 'precision_at_10']:
            self.plot_metric_barplot(metrics_stem[metric], metrics_nostem[metric], metric)

        logging.info("Plotando histograma de R-Precisão...")
        self.plot_r_precision_histogram(metrics_stem['r_precision'], metrics_nostem['r_precision'])

        logging.info("Salvando relatório...")
        self.save_report(metrics_stem, metrics_nostem)


        logging.info("Compactando resultados...")
        self.zip_results()

        end_time = time.time()
        elapsed_time = end_time - start_time
        logging.info(f"Tempo total de execução: {elapsed_time:.2f} segundos")

    def zip_results(self):
        '''
          Compacta todos os arquivos (CSVs e imagens) em um único arquivo.
          Eu fiz isso para que caso seja necessário baixar os arquivos manualmente, você não precisa baixar um a um.
        '''
        with ZipFile(os.path.join(self.data_path, 'RESULTADOS.zip'), 'w') as zipf:
            for foldername, subfolders, filenames in os.walk(self.data_path):
                for filename in filenames:
                    if filename.endswith('.csv') or filename.endswith('.png') or filename.endswith('.CFG'):
                        filepath = os.path.join(foldername, filename)
                        zipf.write(filepath, os.path.relpath(filepath, self.data_path))

analyzer = MetricsAnalyzer()
analyzer.run()
